In [7]:
import torch
from pathlib import Path
from pydicom import dcmread
from PIL import Image
import os
import yaml
import pandas as pd

In [8]:
# Get all the train data samples and test data samples path as list

data_file = Path('data/pneumonia_dataset')

train_path = list(data_file.glob('stage_2_train_images/*.dcm'))
test_path = list(data_file.glob('stage_2_test_images/*.dcm'))
print(f'Train Data Samples: {len(train_path)}')
print(f'Test Data Samples: {len(test_path)}')

Train Data Samples: 26684
Test Data Samples: 3000


In [9]:
# Split the total data of 26684 into train and val dataset for YOLOv8
train_size = 20000


yolo_train_path = train_path[0: 20000]
val_path = train_path[20000:]

print(f'YOLO Train Data Samples: {len(yolo_train_path)}')
print(f'YOLO val Data Samples: {len(val_path)}')

YOLO Train Data Samples: 20000
YOLO val Data Samples: 6684


In [10]:
def dcm_to_jpg(dcm_path, save_path):
    '''
    Convert a DICOM (.dcm) file to a JPEG (.jpg) file and save it.

    This function reads a DICOM file from `dcm_path`, converts the pixel data
    to an image, and saves the image as a JPEG file at `save_path`.

    Args:
        dcm_path (str): File path of the input DICOM file.
        save_path (str): File path to save the output JPEG file.

    Returns:
        None
    '''
    save_path = Path(save_path)

    if not save_path.is_dir():
        dcm_data = dcmread(dcm_path)
        dcm_pixel_array = dcm_data.pixel_array
        image = Image.fromarray(dcm_pixel_array)
        image.save(save_path, 'JPEG')


In [11]:
# Create YOLO File format list

yolo_file_format = ['YOLO_format_data', ['images', 'labels', 'data.yaml'], ['train', 'val']]

# Create the YOLO file format directory
for x in range(2):
    for y in range(2):
        form_path = os.path.join(yolo_file_format[0], yolo_file_format[1][x], yolo_file_format[2][y])
        form_path = Path(form_path)
        form_path.mkdir(parents=True, exist_ok=True)

# # Create data.yaml file
yaml_file = Path(yolo_file_format[0]) / yolo_file_format[1][2]

# Content for the data.yaml file as required by the YOLO model
yaml_content = {
    'train': f'{yolo_file_format[1][0]}/{yolo_file_format[2][0]}',
    'val': f'{yolo_file_format[1][0]}/{yolo_file_format[2][1]}',
    'nc': 1,
    'names': ['pneumonia']
}

# write the content into the data.yaml file
with open(yaml_file, 'w') as file:
    yaml.dump(yaml_content, file)

In [12]:
# Save all the training images into the images direcotry

train_image_name_list = []
for x in range(len(yolo_train_path)):
    img = f'img{x+1}.jpg'
    train_image_name_list.append(img)

train_yolo_img = Path('YOLO_format_data/images/train')



for index, path in enumerate(yolo_train_path):
    save_path = os.path.join(train_yolo_img, train_image_name_list[index])
    dcm_to_jpg(dcm_path=path, save_path=save_path)



In [13]:
# Save all the val images into the val directory

val_image_name_list = []
cont = len(train_image_name_list)
cont
for x in range(len(val_path)):
    img = f'img{cont+x+1}.jpg'
    val_image_name_list.append(img)

val_yolo_img = Path('YOLO_format_data/images/val')

for index, path in enumerate(val_path):
    save_path = os.path.join(val_yolo_img, val_image_name_list[index])
    dcm_to_jpg(dcm_path=path, save_path=save_path)

In [14]:
# File path of the labels from the working data and convert the csv file into a pandas DataFrame

label_file_path = 'data/pneumonia_dataset/stage_2_train_labels.csv'
label_csv = pd.read_csv(label_file_path)
label_csv

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [15]:
# Create a list of all the txt files names for train labels file

train_image_txt_list = []

for image_name in train_image_name_list:
    image_name = Path(image_name)
    name = image_name.stem
    txt_fil_name = f'{name}.txt'
    train_image_txt_list.append(txt_fil_name)

In [16]:
# File path to save the train labels file to
labels_train = Path('YOLO_format_data/labels/train')


# Iterate over each path from the list of training data path
for index, path in enumerate(yolo_train_path):

    # Patient Id of the instance of data 
    id = path.stem
    # Get all the label data associated with the Id
    bbox_data = label_csv[label_csv['patientId'] == id] 

    # final file path to save the instance of label data to
    labels_train_txt = os.path.join(labels_train, train_image_txt_list[index])

    # Create a  label file for each image data
    with open(labels_train_txt, 'w') as f:
        # write label into the file if the bbox for the image exists 
        if bbox_data.iloc[0, 5].item() == 1:
            label_class_id = bbox_data.iloc[0, 5].item()
            for x in range(len(bbox_data)):
                x1 = bbox_data.iloc[x, 1].item() / 1024
                y1 = bbox_data.iloc[x, 2].item() / 1024
                width = bbox_data.iloc[x, 3].item() / 1024
                height = bbox_data.iloc[x, 4].item() / 1024
                f.write(f'{label_class_id-1} {x1} {y1} {width} {height}')
                f.write('\n')
        # Write nth into the file if no bbox for the instance of data exist
        else:
            pass

In [17]:
# Create a list of all the txt files names for val labels file

val_image_txt_list = []

for image_name in val_image_name_list:
    image_name = Path(image_name)
    name = image_name.stem
    txt_fil_name = f'{name}.txt'
    val_image_txt_list.append(txt_fil_name)

In [18]:
labels_val = Path('YOLO_format_data/labels/val')

x = 0
for index, path in enumerate(val_path):
    id = path.stem
    bbox_data = label_csv[label_csv['patientId'] == id]

    labels_train_txt = os.path.join(labels_val, val_image_txt_list[index])

    with open(labels_train_txt, 'w') as f:
        if bbox_data.iloc[0, 5].item() == 1:
            label_class_id = bbox_data.iloc[0, 5].item()
            for x in range(len(bbox_data)):
                x1 = bbox_data.iloc[x, 1].item() / 1024
                y1 = bbox_data.iloc[x, 2].item() / 1024
                width = bbox_data.iloc[x, 3].item() / 1024
                height = bbox_data.iloc[x, 4].item() / 1024
                f.write(f'{label_class_id-1} {x1} {y1} {width} {height}')
                f.write('\n')
        # Write nth into the file if no bbox for the instance of data exist
        else:
            pass

    # break
    

# Workign with the data part is done...now fine tuning the model part


In [19]:
from ultralytics import YOLO

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [21]:
model = YOLO('yolov8n.pt')

In [22]:
train_results = model.train(
    data='YOLO_format_data/data.yaml',
    epochs=5,
    imgsz=640,
    batch=16,
    name='medical_imaging'
)

New https://pypi.org/project/ultralytics/8.3.146 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.145  Python-3.13.1 torch-2.6.0+cu126 CUDA:0 (NVIDIA T400 4GB, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=YOLO_format_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=medical_imaging, nbs=64, nms=False, opset=None, opt

train: Scanning C:\Users\w10195102\Desktop\PROJECTS\ML_Projects\Medical_Image_Classification\YOLO_format_data\labels\train... 20000 images, 14917 backgrounds, 0 corrupt: 100%|██████████| 20000/20000 [00:18<00:00, 1102.88it/s]


train: New cache created: C:\Users\w10195102\Desktop\PROJECTS\ML_Projects\Medical_Image_Classification\YOLO_format_data\labels\train.cache
val: Fast image access  (ping: 0.10.1 ms, read: 4.40.9 MB/s, size: 57.1 KB)


val: Scanning C:\Users\w10195102\Desktop\PROJECTS\ML_Projects\Medical_Image_Classification\YOLO_format_data\labels\val... 6684 images, 5755 backgrounds, 0 corrupt: 100%|██████████| 6684/6684 [00:05<00:00, 1143.52it/s]


val: New cache created: C:\Users\w10195102\Desktop\PROJECTS\ML_Projects\Medical_Image_Classification\YOLO_format_data\labels\val.cache
Plotting labels to c:\Users\w10195102\Desktop\PROJECTS\ML_Projects\runs\detect\medical_imaging\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\w10195102\Desktop\PROJECTS\ML_Projects\runs\detect\medical_imaging
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       6684       1359      0.007       0.71     0.0475     0.0153

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/5      3.84G      2.185      3.388       2.17         11        640: 100%|██████████| 1250/1250 [22:09<00:00,  1.06s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 209/209 [03:09<00:00,  1.10it/s]


                   all       6684       1359     0.0674      0.231     0.0525     0.0172

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/5      3.84G        2.1      3.194        2.1         14        640:  78%|███████▊  | 973/1250 [16:28<04:41,  1.02s/it]


KeyboardInterrupt: 

In [ ]:
# len(yolo_train_path)
# x = dcmread(yolo_train_path[3])
# shape = x.pixel_array.shape
# for x in shape:
#     print(x)


# for index, path in enumerate(yolo_train_path):
#     data = dcmread(path)
#     image_shape = data.pixel_array.shape
#     for x in image_shape:
#         if x != 1024:
#             print(index)
#             break

1024
1024


Great — since you're training YOLOv8 in a Python notebook using the Ultralytics library, here's exactly how to safely stop training and resume it later.

✅ How to Stop Training Safely
You can interrupt the notebook kernel (or click the stop button), and YOLOv8 will automatically save the last checkpoint.

By default, it saves to:

swift
Copy
Edit
runs/detect/train/weights/last.pt
✅ How to Resume Training Later
In a new cell, just run:

python
Copy
Edit
from ultralytics import YOLO

# Load from the last saved checkpoint
model = YOLO('runs/detect/train/weights/last.pt')

# Resume training
model.train(resume=True)
This resumes:

Optimizer state

Epoch count

Best weights

All hyperparameters

🔁 Optional: Resume with Adjustments
Want to change something (e.g., number of total epochs)?

python
Copy
Edit
model.train(resume=True, epochs=100)
YOLO will pick up where it left off and continue up to 100 total epochs.

📁 Where Your Checkpoints Are Saved
Each training run creates a directory like:

bash
Copy
Edit
runs/detect/train/  # or train2, train3, etc.
Inside that, you'll see:

weights/last.pt – the latest checkpoint

weights/best.pt – the best performing model so far

results.csv – your training logs

💡 Pro Tip: Set a Custom Save Path
To avoid confusion across multiple runs:

python
Copy
Edit
model.train(data='data.yaml', epochs=100, name='rsna_pneumonia_run1')
This saves to:

bash
Copy
Edit
runs/detect/rsna_pneumonia_run1/
Let me know if you'd like to auto-resume from the best model or evaluate your current checkpoint!

In [4]:
# Continue training the mdoel form where it left of last time

from ultralytics import YOLO

model = YOLO('../runs/detect/medical_imaging/weights/last.pt')

In [ ]:
model.train(resume=True)

In [8]:

model = YOLO('../runs/detect/medical_imaging/weights/last.pt')

model.train(data='YOLO_format_data/data.yaml', epochs=5, project='runs/detect', name='medical_imaging_v2')


New https://pypi.org/project/ultralytics/8.3.146 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.145  Python-3.13.1 torch-2.6.0+cu126 CUDA:0 (NVIDIA T400 4GB, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=YOLO_format_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../runs/detect/medical_imaging/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=medical_imaging

train: Scanning C:\Users\w10195102\Desktop\PROJECTS\ML_Projects\Medical_Image_Classification\YOLO_format_data\labels\train.cache... 20000 images, 14917 backgrounds, 0 corrupt: 100%|██████████| 20000/20000 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 240.462.9 MB/s, size: 57.1 KB)


val: Scanning C:\Users\w10195102\Desktop\PROJECTS\ML_Projects\Medical_Image_Classification\YOLO_format_data\labels\val.cache... 6684 images, 5755 backgrounds, 0 corrupt: 100%|██████████| 6684/6684 [00:00<?, ?it/s]


Plotting labels to runs\detect\medical_imaging_v22\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\medical_imaging_v22
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      3.89G      1.905       2.85      1.951         14        640: 100%|██████████| 1250/1250 [23:04<00:00,  1.11s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 209/209 [03:35<00:00,  1.03s/it]


                   all       6684       1359      0.142      0.199     0.0795     0.0263

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/5      3.99G      1.947      2.856       1.95          6        640:   2%|▏         | 19/1250 [00:22<24:12,  1.18s/it]


KeyboardInterrupt: 